In [35]:
import sys

In [ ]:

#!pip install arrow

In [1]:
# your id
ym_app_id = 'a746048edf594f238083f56a028da76a'

In [2]:
code_request = f'https://oauth.yandex.ru/authorize?response_type=token&client_id={ym_app_id}'

In [3]:

#print("Перейти по ссылке:")
print(code_request)

https://oauth.yandex.ru/authorize?response_type=token&client_id=a746048edf594f238083f56a028da76a


In [4]:
if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

In [5]:
import requests
import json
import arrow
import datetime
import sqlalchemy
from requests.exceptions import ConnectionError
from time import sleep
from datetime import date, timedelta
from time import time
from sqlalchemy import create_engine
from sqlalchemy import Float,String,Integer
import pandas as pd


In [6]:
AgencyClientsURL = 'https://api.direct.yandex.com/json/v5/agencyclients'

In [7]:
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

In [8]:
AmountURL = 'https://api.direct.yandex.ru/live/v4/json/'

In [9]:
#insert token
token = 'AgAAAAAIQ2KfAASlbGLfazAXlEZppm2H0IX-YmE'


In [10]:
headers = {"Authorization": "Bearer " + token,  # OAuth-токен. Использование слова Bearer обязательно
           #"Client-Login": clientLogin,  # Логин клиента рекламного агентства
           "Accept-Language": "ru",  # Язык ответных сообщений
           }

In [11]:
AgencyClientsBody = {
    "method": "get",
    "params": {
        "SelectionCriteria": {
            "Archived": "NO"   # Получить только активных клиентов
        },
        "FieldNames": ["Login"],
        "Page": {
            "Limit": 10000,  # Получить не более 10000 клиентов в ответе сервера
            "Offset": 0
        }
    }
}

In [12]:
HasAllClientLoginsReceived = False
ClientList = []

In [13]:
while not HasAllClientLoginsReceived:
    ClientsResult = requests.post(AgencyClientsURL, json.dumps(AgencyClientsBody), headers=headers).json()
    for Client in ClientsResult['result']['Clients']:
        ClientList.append(Client["Login"])
    if ClientsResult['result'].get("LimitedBy", False):
        AgencyClientsBody['Page']['Offset'] = ClientsResult['result']["LimitedBy"]
    else:
        HasAllClientLoginsReceived = True

Получаем расход за предыдущие семь дней

In [14]:
today = datetime.date.today()
today_minus_7_days = timedelta(days = 7)

In [15]:
body_cost = {
    "params": {
        "SelectionCriteria": {
            "DateFrom": str(today - today_minus_7_days),
             "DateTo": str(today)
        },
        "FieldNames": [
            "Cost"
        ],
        "ReportName": u("Direct_account_cost"), #"ACCOUNT_PERFORMANCE"
        "ReportType": "ACCOUNT_PERFORMANCE_REPORT",
        "DateRangeType": "CUSTOM_DATE",
        "Format": "TSV",
        "IncludeVAT": "NO",
        "IncludeDiscount": "NO"
    }
}

In [16]:
resultcsv = "Login;Costs\n"

In [17]:
headers['skipReportHeader'] = "true"
headers['skipColumnHeader'] = "true"
headers['skipReportSummary'] = "true"
headers['returnMoneyInMicros'] = "false"

In [18]:
for Client in ClientList:
    # Добавление HTTP-заголовка "Client-Login"
    headers['Client-Login'] = Client
    # Кодирование тела запроса в JSON
    requestBody = json.dumps(body_cost, indent=4)
    # Запуск цикла для выполнения запросов
    # Если получен HTTP-код 200, то содержание отчета добавляется к результирующим данным
    # Если получен HTTP-код 201 или 202, выполняются повторные запросы
    while True:
        try:
            req = requests.post(ReportsURL, requestBody, headers=headers)
            req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
            if req.status_code == 400:
                break
            elif req.status_code == 200:
                if req.text != "":
                    tempresult = req.text.split('\t')
                    resultcsv += "{};{}\n".format(Client,str(tempresult[0]).replace('.', ','))
                else:
                    resultcsv += "{};0\n".format(Client)
                break
            elif req.status_code == 201:
                retryIn = int(req.headers.get("retryIn", 60))
                sleep(retryIn)
            elif req.status_code == 202:
                retryIn = int(req.headers.get("retryIn", 60))
                sleep(retryIn)
            elif req.status_code == 500:
                break
            elif req.status_code == 502:
                break
            else:
                break

        # Обработка ошибки, если не удалось соединиться с сервером API Директа
        except ConnectionError:
            break

        # Если возникла какая-либо другая ошибка
        except:
            break

# Создание и запись файла
filename = "AccountsPerfomanceReport_{}.csv".format(str(time()))
resultfile = open(filename, 'w+')
resultfile.write(resultcsv)
resultfile.close()

In [19]:
# for Client in ClientList[:10]:
#     # Добавление HTTP-заголовка "Client-Login"
#     headers['Client-Login'] = Client
#     # Кодирование тела запроса в JSON
#     requestBody = json.dumps(body_cost, indent=4)
#     # Запуск цикла для выполнения запросов
#     # Если получен HTTP-код 200, то содержание отчета добавляется к результирующим данным
#     # Если получен HTTP-код 201 или 202, выполняются повторные запросы
#     while True:
#         try:
#             req = requests.post(ReportsURL, requestBody, headers=headers)
#             req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
#             if req.status_code == 400:
#                 print("Параметры запроса указаны неверно или достугнут лимит отчетов в очереди")
#                 print("RequestId: {}".format(req.headers.get("RequestId", False)))
#                 print("JSON-код запроса: {}".format(u(body_cost)))
#                 print("JSON-код ответа сервера: \n{}".format(u(req.json())))
#                 break
#             elif req.status_code == 200:
#                 print("Отчет для аккаунта {} создан успешно".format(str(Client)))
#                 print("RequestId: {}".format(req.headers.get("RequestId", False)))
#                 if req.text != "":
#                     tempresult = req.text.split('\t')
#                     resultcsv += "{};{}\n".format(Client,str(tempresult[0]).replace('.', ','))
#                 else:
#                     resultcsv += "{};0\n".format(Client)
#                 break
#             elif req.status_code == 201:
#                 print("Отчет для аккаунта {} успешно поставлен в очередь в режиме offline".format(str(Client)))
#                 retryIn = int(req.headers.get("retryIn", 60))
#                 print("Повторная отправка запроса через {} секунд".format(retryIn))
#                 print("RequestId: {}".format(req.headers.get("RequestId", False)))
#                 sleep(retryIn)
#             elif req.status_code == 202:
#                 print("Отчет формируется в режиме офлайн".format(str(Client)))
#                 retryIn = int(req.headers.get("retryIn", 60))
#                 print("Повторная отправка запроса через {} секунд".format(retryIn))
#                 print("RequestId: {}".format(req.headers.get("RequestId", False)))
#                 sleep(retryIn)
#             elif req.status_code == 500:
#                 print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее.")
#                 print("RequestId: {}".format(req.headers.get("RequestId", False)))
#                 print("JSON-код ответа сервера: \n{}".format(u(req.json())))
#                 break
#             elif req.status_code == 502:
#                 print("Время формирования отчета превысило серверное ограничение.")
#                 print(
#                     "Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
#                 print("JSON-код запроса: {}".format(body))
#                 print("RequestId: {}".format(req.headers.get("RequestId", False)))
#                 print("JSON-код ответа сервера: \n{}".format(u(req.json())))
#                 break
#             else:
#                 print("Произошла непредвиденная ошибка")
#                 print("RequestId: {}".format(req.headers.get("RequestId", False)))
#                 print("JSON-код запроса: {}".format(body))
#                 print("JSON-код ответа сервера: \n{}".format(u(req.json())))
#                 break

#         # Обработка ошибки, если не удалось соединиться с сервером API Директа
#         except ConnectionError:
#             # В данном случае мы рекомендуем повторить запрос позднее
#             print("Произошла ошибка соединения с сервером API")
#             # Принудительный выход из цикла
#             break

#         # Если возникла какая-либо другая ошибка
#         except:
#             # В данном случае мы рекомендуем проанилизировать действия приложения
#             print("Произошла непредвиденная ошибка")
#             # Принудительный выход из цикла
#             break

# print("Создание отчетов для аккаунтов завершено")

# # Создание и запись файла
# filename = "AccountsPerfomanceReport_{}.csv".format(str(time()))
# resultfile = open(filename, 'w+')
# resultfile.write(resultcsv)
# resultfile.close()

# print("Результат записан в файл {}".format(filename))

In [20]:
df_cost = pd.read_csv(filename,sep=';',skip_blank_lines=True,error_bad_lines=False)

Получаем бюджет

In [21]:
data_amount_1batch = {
    "method" : "AccountManagement",
    "token" : token,
    "param" : {
        "Action": "Get",
        "SelectionCriteria" : {
            "Logins" : ClientList[:50]
        }
                    
    }
}

In [22]:
data_amount_2batch = {
    "method" : "AccountManagement",
    "token" : token,
    "param" : {
        "Action": "Get",
        "SelectionCriteria" : {
            "Logins" : ClientList[50:]
        }
                    
    }
}

In [23]:
jdata1 = json.dumps(data_amount_1batch, ensure_ascii=False).encode('utf8')
jdata2 = json.dumps(data_amount_2batch, ensure_ascii=False).encode('utf8')

In [24]:
response1 = requests.post(AmountURL,jdata1)
response2 = requests.post(AmountURL,jdata2)

In [25]:
res1 = response1.json()
res2 = response2.json()

In [26]:
df_amount = pd.DataFrame(columns=['Login','Amount'])

In [27]:
for i in res1['data']['Accounts']:
    df_amount = df_amount.append({'Login':i['Login'],\
                    'Amount': i['Amount']\
                    },ignore_index=True)

In [28]:
for i in res2['data']['Accounts']:
    df_amount = df_amount.append({'Login':i['Login'],\
                    'Amount': i['Amount']\
                    },ignore_index=True)

In [29]:
df_cost['Costs'] = df_cost['Costs'].str.replace(',' , '.')

In [30]:
End_df = pd.merge(df_amount,df_cost,on='Login',how='outer')

In [31]:
End_df

,Login,Amount,Costs
0,novostroy-spb2012,1689318.14,0
1,pst-sochi,4852.15,147.85
2,novoseli-ru,55646.12,8764.32
3,novoseli-spb,5791.77,1596.89
4,comestate-ru,0,0
5,ocenim-kvartiru,8454.3,0
6,client-mpn-cre,165764.79,39615.76
7,caricyno2,3535.19,0.00
8,landing-spb,364933.81,413739.10
9,pestovo-club,4184.76,0.00


In [32]:
#Пишем в Базу

In [33]:
db_credentials = {
    'host':     '127.0.0.1',
    'user':     "postgres",
    'password': "Filipievnew",
    'db':       'Mediapronet',
    'port':     "5432",
}

conn_string = 'postgresql://{user}:{password}@{host}:{port}/{db}'.format(**db_credentials)
engine = create_engine(conn_string, echo=True)

In [34]:
End_df.to_sql('Direct',\
          con=engine,\
          schema='public',\
          if_exists='replace',\
          index=False,\
          dtype={"Login": String,"Amount": Float ,"Costs": Float})

2019-08-08 16:55:11,462 INFO sqlalchemy.engine.base.Engine select version()
2019-08-08 16:55:11,463 INFO sqlalchemy.engine.base.Engine {}
2019-08-08 16:55:11,464 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-08-08 16:55:11,465 INFO sqlalchemy.engine.base.Engine {}
2019-08-08 16:55:11,467 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-08-08 16:55:11,468 INFO sqlalchemy.engine.base.Engine {}
2019-08-08 16:55:11,469 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-08-08 16:55:11,470 INFO sqlalchemy.engine.base.Engine {}
2019-08-08 16:55:11,471 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-08-08 16:55:11,471 INFO sqlalchemy.engine.base.Engine {}
2019-08-08 16:55:11,473 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2019-08-08 16:55:11

2019-08-08 16:55:11,519 INFO sqlalchemy.engine.base.Engine {'table_oid': 16465}
2019-08-08 16:55:11,521 INFO sqlalchemy.engine.base.Engine 
DROP TABLE public."Direct"
2019-08-08 16:55:11,522 INFO sqlalchemy.engine.base.Engine {}
2019-08-08 16:55:11,525 INFO sqlalchemy.engine.base.Engine COMMIT
2019-08-08 16:55:11,530 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE public."Direct" (
	"Login" VARCHAR, 
	"Amount" FLOAT, 
	"Costs" FLOAT
)


2019-08-08 16:55:11,531 INFO sqlalchemy.engine.base.Engine {}
2019-08-08 16:55:11,539 INFO sqlalchemy.engine.base.Engine COMMIT
2019-08-08 16:55:11,541 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-08 16:55:11,542 INFO sqlalchemy.engine.base.Engine INSERT INTO public."Direct" ("Login", "Amount", "Costs") VALUES (%(Login)s, %(Amount)s, %(Costs)s)
2019-08-08 16:55:11,542 INFO sqlalchemy.engine.base.Engine ({'Login': 'novostroy-spb2012', 'Amount': '1689318.14', 'Costs': '0'}, {'Login': 'pst-sochi', 'Amount': '4852.15', 'Costs': '147.85'}, {'